# ДОМАШНЕЕ ЗАДАНИЕ № 4. Рекомендательные системы и Spark MLlib
---
**Дисциплина:** Методы обработки больших данных  
**Студент:** Голдышев Д.М. (goldyshev02@mail.ru)  
**Группа:** ИУ6-31М  

## Задание 3. Факторизация матрицы
---
### Цель
Построить и сравнить рекомендательные модели — **ALS** и **Item-based Collaborative Filtering** — с использованием **Spark MLlib** и **DataFrame API**.

### Задачи
1. Выбрать модель `ALS` по минимальному значению `RMSE` с помощью `k-fold` кросс-валидации (`k=4`).
2. Параметры для перебора:
    - Количество факторов: `[5, 10, 15]`;
    - Регуляризация: `[0.001, 0.01, 0.1, 1, 10]`.
3. Сравнить результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов.
4. Выполнить анализ на двух датасетах: `ml-latest-small`, затем `ml-latest`.

### Примечание
В рамках задания, а именно в контексте необходимости *"сравнить результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов"*, было принято решение считать:
- под «коллаборативной фильтрацией» — **Item-based CF**;
- под «факторизацией матрицы» — **модель ALS**.

Сравнение этих подходов позволяет сопоставить **базовый метод** (item-based CF, рекомендации по похожим фильмам) с **модельным методом** (ALS, латентные факторы).

## Шаг 1. Импорт зависимостей и конфигурация
---

In [1]:
import os
import sys
import time
import tempfile
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Optional

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, 
    IntegerType, LongType, FloatType, DoubleType, StringType
)
from pyspark import StorageLevel

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Только для RankingMetrics
from pyspark.mllib.evaluation import RankingMetrics

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Путь к директории с данными MovieLens
DATA_DIR_FULL = "/app/data/ml-latest"
# Путь к директории с данными MovieLens (small)
DATA_DIR_SMALL = "/app/data/ml-latest-small"

# Гиперпараметры ALS
RANKS = [5, 10, 15]                         # Количество латентных факторов
REG_PARAMS = [0.001, 0.01, 0.1, 1.0, 10.0]  # Регуляризация L2
MAX_ITER = 15                               # Максимум итераций ALS

# Параметры кросс-валидации
NUM_FOLDS = 4  # k-fold

# Параметры Item-based CF
MIN_COOC_COUNT = 5          # Минимальное число общих пользователей для расчёта сходства
TOP_N_NEIGHBORS = 50        # Сколько соседей хранить для каждого фильма
TOP_K_RECOMMENDATIONS = 10  # Топ-K рекомендаций для оценки Ranking-метрик

# Порог релевантности
RELEVANCE_THRESHOLD = 3.5  # Рейтинг >= 3.5 считается "релевантным" для Ranking-метрик

# Для воспроизводимости:
RANDOM_SEED = 23

# Функция для печати "заголовков"
def print_header(header: str):
    print("=" * 60)
    print(header)
    print("=" * 60)

### Проверка существования данных

In [2]:
def check_file(path: str):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"{path}: файл не найден")
    try:
        with open(path, "r", encoding="utf-8") as f:
            f.read(1)  # пробное чтение
    except Exception as e:
        raise IOError(f"{path}: файл существует, но не читается: {e}")

def check_dataset_files(data_dir: str) -> None:
    required_files = ["ratings.csv", "movies.csv", "links.csv", "tags.csv"]
    for fname in required_files:
        check_file(os.path.join(data_dir, fname))

try:
    check_dataset_files(DATA_DIR_SMALL)
    # Т.к. всё равно пока что не можем работать с большим объемом данных
    # check_dataset_files(DATA_DIR_FULL)
    print("Файлы, необходимые для работы, найдены и читаются.")
except Exception as e:
    print("Ошибка при проверке файлов:")
    print(e)
    raise SystemExit(1)

Файлы, необходимые для работы, найдены и читаются.


## Шаг 2. Инициализация SparkSession
---

In [3]:
def create_spark_session(app_name: str) -> SparkSession:    
    spark = (
        SparkSession.builder
        .appName(app_name)
        # Включаем Adaptive Query Execution (для динамического перепланирования запроса во время выполнения)
        .config("spark.sql.adaptive.enabled", "true")
        # Для автоматического сливания слишком маленьких партиций после shuffle
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
        # Broadcast threshold для небольших таблиц (которые целиком рассылаются на все executors)
        .config("spark.sql.autoBroadcastJoinThreshold", "50MB")
        # Сериализация Kryo (пробуем более быструю и компактную сериализация объектов)
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .getOrCreate()
    )
    # Количество партиций при shuffle и настройки памяти указаны на уровне Docker-контейнера
    return spark

# Создаём SparkSession
spark = create_spark_session("HW4_ALS")
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 20:07:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/10 20:07:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/10 20:07:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Шаг 3. Загрузка данных и определение схем
---

In [4]:
# Явно задаем типы для надежности
RATINGS_SCHEMA = StructType([
    StructField("userId", IntegerType(), nullable=False),
    StructField("movieId", IntegerType(), nullable=False),
    StructField("rating", FloatType(), nullable=False),
    StructField("timestamp", LongType(), nullable=True)  # Unix timestamp
])
MOVIES_SCHEMA = StructType([
    StructField("movieId", IntegerType(), nullable=False),
    StructField("title", StringType(), nullable=True),
    StructField("genres", StringType(), nullable=True)  # pipe-separated
])
LINKS_SCHEMA = StructType([
    StructField("movieId", IntegerType(), nullable=False),
    StructField("imdbId", StringType(), nullable=True),
    StructField("tmdbId", IntegerType(), nullable=True)
])
TAGS_SCHEMA = StructType([
    StructField("userId", IntegerType(), nullable=False),
    StructField("movieId", IntegerType(), nullable=False),
    StructField("tag", StringType(), nullable=True),
    StructField("timestamp", LongType(), nullable=True)
])

def load_movielens_data(data_dir: str, spark: SparkSession) -> dict:
    """
    Загружает все файлы датасета MovieLens.
    Args:
        data_dir: Путь к директории с данными
        spark: SparkSession
    Returns:
        Словарь с DataFrame: ratings, movies, links, tags
    """
    start_time = time.time()
    ratings_df = (
        spark.read
        .option("header", "true")  # Первая строка — заголовок
        .schema(RATINGS_SCHEMA)    # Явная схема
        .csv(f"{data_dir}/ratings.csv")
    )
    movies_df = (
        spark.read
        .option("header", "true")
        .schema(MOVIES_SCHEMA)
        .csv(f"{data_dir}/movies.csv")
    )
    links_df = (
        spark.read
        .option("header", "true")
        .schema(LINKS_SCHEMA)
        .csv(f"{data_dir}/links.csv")
    )
    tags_df = (
        spark.read
        .option("header", "true")
        .schema(TAGS_SCHEMA)
        .csv(f"{data_dir}/tags.csv")
    )
    elapsed = time.time() - start_time
    print(f"+ Данные из директории '{data_dir}' загружены за {elapsed:.2f} сек")
    return {
        "ratings": ratings_df,
        "movies": movies_df,
        "links": links_df,
        "tags": tags_df
    }


def validate_data(data: dict) -> None:
    """
    Проверяет корректность загруженных данных.
    Args:
        data: Словарь с DataFrame
    Raises:
        AssertionError: Если данные некорректны
    """
    ratings_df = data["ratings"]
    movies_df = data["movies"]
    
    # Проверка: ratings не пустой
    ratings_count = ratings_df.count()
    assert ratings_count > 0, "ratings.csv пустой!"
    
    # Проверка: типы столбцов ratings
    assert ratings_df.schema["userId"].dataType == IntegerType(), "userId должен быть IntegerType"
    assert ratings_df.schema["movieId"].dataType == IntegerType(), "movieId должен быть IntegerType"
    assert ratings_df.schema["rating"].dataType == FloatType(), "rating должен быть FloatType"
    
    # Проверка: рейтинги в диапазоне [0.5, 5.0]
    rating_stats = ratings_df.select(
        F.min("rating").alias("min_rating"),
        F.max("rating").alias("max_rating")
    ).collect()[0]
    assert rating_stats["min_rating"] >= 0.5, f"Минимальный рейтинг < 0.5: {rating_stats['min_rating']}"
    assert rating_stats["max_rating"] <= 5.0, f"Максимальный рейтинг > 5.0: {rating_stats['max_rating']}"
    
    # Проверка: нет NULL в ключевых полях
    null_ratings = ratings_df.filter(
        F.col("userId").isNull() | F.col("movieId").isNull() | F.col("rating").isNull()
    ).count()
    assert null_ratings == 0, f"Найдено {null_ratings} записей с NULL в ratings"
    
    # Проверка: movies не пустой
    movies_count = movies_df.count()
    assert movies_count > 0, "movies.csv пустой!"

    print("+ Валидация данных пройдена успешно")


# Загружаем данные ml-latest-small
data_small = load_movielens_data(DATA_DIR_SMALL, spark)
validate_data(data_small)

# Кэшируем для многократного использования
ratings_small = data_small["ratings"].cache()
movies_small = data_small["movies"].cache()

# Статистика
print(f"\nРазмеры датасета ml-latest-small:")
print(f"  Рейтингов:         {ratings_small.count():,}")
print(f"  Пользователей:     {ratings_small.select('userId').distinct().count():,}")
print(f"  Оцененных фильмов: {ratings_small.select('movieId').distinct().count():,}")
print(f"  Всего фильмов:     {movies_small.count():,}")

+ Данные из директории '/app/data/ml-latest-small' загружены за 0.66 сек
+ Валидация данных пройдена успешно

Размеры датасета ml-latest-small:
  Рейтингов:         100,836
  Пользователей:     610
  Оцененных фильмов: 9,724
  Всего фильмов:     9,742


## Шаг 4. Подготовка данных и разбиение на train/test
---

In [5]:
def prepare_train_test_split(
    ratings_df,
    train_ratio: float = 0.8,
    seed: int = RANDOM_SEED
) -> Tuple:
    """
    Разбивает данные на обучающую и тестовую выборки.    
    Args:
        ratings_df: DataFrame с рейтингами
        train_ratio: Доля обучающей выборки
        seed: Seed для воспроизводимости
    Returns:
        (train_df, test_df)
    """
    train_df, test_df = ratings_df.randomSplit(
        [train_ratio, 1 - train_ratio], 
        seed=seed
    )
    return train_df, test_df

# Разбиваем данные: 80% train, 20% test
train_small, test_small = prepare_train_test_split(ratings_small)

# Кэшируем для многократного использования в CV
train_small = train_small.cache()
test_small = test_small.cache()

print(f"Разбиение данных ml-latest-small:")
print(f"  Train: {train_small.count():,} рейтингов")
print(f"  Test:  {test_small.count():,} рейтингов")

Разбиение данных ml-latest-small:
  Train: 80,467 рейтингов
  Test:  20,369 рейтингов


## Шаг 5. ALS: Кросс-валидация с подбором гиперпараметров

In [6]:
def train_als_with_cv(
    train_df,
    ranks: List[int] = RANKS,
    reg_params: List[float] = REG_PARAMS,
    max_iter: int = MAX_ITER,
    num_folds: int = NUM_FOLDS,
    seed: int = RANDOM_SEED
) -> Tuple:
    """
    Обучает ALS с кросс-валидацией и подбором гиперпараметров.
    Args:
        train_df: Обучающий DataFrame
        ranks: Список значений количества факторов
        reg_params: Список значений регуляризации
        max_iter: Максимальное число итераций
        num_folds: Число фолдов для CV
        seed: Seed для воспроизводимости
    Returns:
        (best_model, cv_model, cv_results_df)
    """
    print_header("ОБУЧЕНИЕ ALS С КРОСС-ВАЛИДАЦИЕЙ")
    print(f"Параметры поиска:")
    print(f"  Ranks:      {ranks}")
    print(f"  RegParams:  {reg_params}")
    print(f"  MaxIter:    {max_iter}")
    print(f"  NumFolds:   {num_folds}")
    print(f"  Комбинаций: {len(ranks) * len(reg_params)}")
    
    start_time = time.time()

    # Создаём модель ALS с базовыми параметрами
    als = ALS(
        userCol="userId",           # Столбец с ID пользователей
        itemCol="movieId",          # Столбец с ID фильмов
        ratingCol="rating",         # Столбец с рейтингами
        maxIter=max_iter,           # Максимум итераций
        coldStartStrategy="drop",   # "Холодные" userId/movieId будут отброшены при вычислении метрик
        implicitPrefs=False,        # У нас "явные рейтинги", не интерпретируем рейтинг как "силу сигнала"
        nonnegative=False,          # Разрешаем отрицательные факторы
        seed=seed                   # Фиксируем seed
    )
    
    # Сетка гиперпараметров
    param_grid = (
        ParamGridBuilder()
            .addGrid(als.rank, ranks)           # Количество латентных факторов
            .addGrid(als.regParam, reg_params)  # L2 регуляризация
            .build()
    )
    
    # Настраиваем Evaluator для RMSE, определеляем, как мы измеряем "качество"
    evaluator = RegressionEvaluator(
        metricName="rmse",          # Считаем корень из средней квадратичной ошибки
        labelCol="rating",          # Считаем за "правду"
        predictionCol="prediction"  # Считаем за "предсказание"
    )
    
    # Настраиваем автоматический перебор моделей + кросс-валидация с k-fold
    cv = CrossValidator(
        estimator=als,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        numFolds=num_folds,
        seed=seed,
        parallelism=2  # Обучаем параллельно (т.к. ресурсов немного, то ставим )
    )
     
    # Запуск кросс-валидации
    cv_model = cv.fit(train_df)
    
    elapsed = time.time() - start_time
    print(f"Обучение завершено за {elapsed:.1f} сек ({elapsed/60:.1f} мин)")
    
    # Извлекаем результаты CV
    # avgMetrics — средний RMSE для каждой комбинации параметров
    avg_metrics = cv_model.avgMetrics
    
    # Собираем результаты в DataFrame для анализа
    cv_results = []
    for i, params in enumerate(param_grid):
        rank = params[als.rank]
        reg_param = params[als.regParam]
        rmse = avg_metrics[i]
        cv_results.append({
            "rank": rank,
            "regParam": reg_param,
            "avg_rmse": rmse
        })

    cv_results_df = pd.DataFrame(cv_results)
    cv_results_df = cv_results_df.sort_values("avg_rmse")
    
    # Лучшая модель
    best_model = cv_model.bestModel
    best_rank = best_model.rank
    best_reg_param = best_model._java_obj.parent().getRegParam()
    best_rmse = cv_results_df["avg_rmse"].min()
    
    print_header("РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ")
    print(cv_results_df.to_string(index=False))

    print_header("ЛУЧШАЯ МОДЕЛЬ")
    print(f"  Rank:     {best_rank}")
    print(f"  RegParam: {best_reg_param}")
    print(f"  Avg RMSE: {best_rmse:.6f}")

    return best_model, cv_model, cv_results_df

# Запускаем кросс-валидацию на небольшом датасете
best_als_small, cv_model_small, cv_results_small = train_als_with_cv(train_small)

ОБУЧЕНИЕ ALS С КРОСС-ВАЛИДАЦИЕЙ
Параметры поиска:
  Ranks:      [5, 10, 15]
  RegParams:  [0.001, 0.01, 0.1, 1.0, 10.0]
  MaxIter:    15
  NumFolds:   4
  Комбинаций: 15


25/12/10 20:07:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Обучение завершено за 86.5 сек (1.4 мин)
РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ
 rank  regParam  avg_rmse
    5     0.100  0.913790
   15     0.100  0.913995
   10     0.100  0.917735
    5     0.010  1.110020
   10     0.010  1.229122
   15     0.010  1.293958
    5     1.000  1.324957
   15     1.000  1.324958
   10     1.000  1.324958
    5     0.001  1.344002
   10     0.001  1.465318
   15     0.001  1.565245
    5    10.000  3.664597
   10    10.000  3.664597
   15    10.000  3.664597
ЛУЧШАЯ МОДЕЛЬ
  Rank:     5
  RegParam: 0.1
  Avg RMSE: 0.913790


## Шаг 6. Оценка ALS на тестовой выборке
---

In [7]:
def evaluate_als_on_test(model, test_df) -> dict:
    print_header("ОЦЕНКА ALS НА ТЕСТЕ")
    start_time = time.time()
    
    # Предсказания на тесте (coldStartStrategy="drop" уже уберёт холодные пары)
    predictions = model.transform(test_df)

    # Считаем, сколько потеряли
    n_test = test_df.count()
    n_pred = predictions.count()
    n_cold = n_test - n_pred
    cold_pct = (n_cold / n_test * 100) if n_test > 0 else 0.0
    
    print(f"Предсказания:")
    print(f"  Строк в test:        {n_test:,}")
    print(f"  Строк с prediction:  {n_pred:,}")
    print(f"  Потеряно (cold):     {n_cold:,} ({cold_pct:.2f}%)")
    
    evaluator = RegressionEvaluator(
        metricName="rmse",
        labelCol="rating",
        predictionCol="prediction"
    )
    rmse = evaluator.evaluate(predictions)

    elapsed = time.time() - start_time
    
    print(f"RMSE: {rmse:.6f}")
    print(f"Время оценки: {elapsed:.2f} сек")
    
    return {
        "rmse": rmse,
        "n_predictions": n_pred,
        "n_cold_start": n_cold
    }

# Оцениваем лучшую модель на тесте
als_test_metrics_small = evaluate_als_on_test(
    best_als_small, 
    test_small
)

ОЦЕНКА ALS НА ТЕСТЕ
Предсказания:
  Строк в test:        20,369
  Строк с prediction:  19,563
  Потеряно (cold):     806 (3.96%)
RMSE: 0.881034
Время оценки: 0.33 сек


### Ranking-метрики для ALS

In [8]:
# Для сокращения кода выделяем "пустые" метрики
ZERO_METRICS = {
    "k": TOP_K_RECOMMENDATIONS,
    "n_users": 0,
    "precision_at_k": 0.0,
    "recall_at_k": 0.0,
    "map_at_k": 0.0,
    "ndcg_at_k": 0.0
}

def compute_ranking_metrics_als_both(
    model,
    train_df,
    test_df,
    k: int = TOP_K_RECOMMENDATIONS,
    relevance_threshold: float = RELEVANCE_THRESHOLD
) -> dict:
    """
    Вычисляет ranking-метрики ALS в двух режимах:
        1. "all" - насколько хорошо ALS способен угадывать все релевантные
           фильмы в тесте, включая те, которых он никогда не видел?
        2. "warm" - насколько хорошо ALS способен попасть в релевантные
           фильмы среди тех, которые он хотя бы видел на обучении?
    Args:
        model:     Обученная ALS-модель
        train_df:  Обучающая выборка
        test_df:   Тестовая выборка
        k:         Число рекомендаций K в метриках
        relevance_threshold: Порог релевантности
    Returns:
        Словарь с метриками под каждый режим
    """
    
    def _compute_single_mode(mode: str) -> dict:
        """
        Считает метрики для одного режима.
        Args:
            mode: Режим ("all", "warm")
        Returns:
            Словарь с метриками для выбранного режима
        """
        assert mode in {"all", "warm"}
        print_header(f"RANKING-МЕТРИКИ ALS @K={k} | mode = '{mode}'")
    
        start_time = time.time()
        
        # Базовый ground truth - релевантные фильмы из test
        ground_truth_base = test_df.filter(F.col("rating") >= relevance_threshold)
        # В режиме "warm" ограничиваемся фильмами из train
        if mode == "warm":
            # Все фильмы, которые модель "видела" при обучении
            train_movies = train_df.select("movieId").distinct()
            # Оставляем только такие фильмы в ground truth
            ground_truth_base = ground_truth_base.join(
                train_movies, on="movieId", how="inner"
            )
        # Теперь строим для каждого пользователя список релевантных фильмов
        ground_truth = (
            ground_truth_base
                .groupBy("userId")
                .agg(F.collect_set("movieId").alias("relevant_movies"))
        )
        
        # Рекомендации ALS: топ-K фильмов для всех пользователей из теста
        test_users = test_df.select("userId").distinct()
        recommendations = model.recommendForUserSubset(test_users, k)
        # Оставляем userId и список рекомендованных movieId (без score)
        recommendations_flat = (
            recommendations
                .select(
                    "userId",
                    F.col("recommendations.movieId").alias("recommended_movies")
                )
        )
        
        # Соединяем рекомендации и ground truth по userId
        joined = recommendations_flat.join(ground_truth, on="userId", how="inner")
        # Если ни одного пользователя не осталось, возвращаем нули
        if joined.rdd.isEmpty():
            print("! Нет пользователей для оценки в режиме", mode)
            return ZERO_METRICS
        
        # Преобразуем к формату RDD для RankingMetrics
        prediction_and_labels_rdd = (
            joined
                .select("recommended_movies", "relevant_movies")
                .rdd
                .map(lambda row: (row.recommended_movies, list(row.relevant_movies)))
        )
        
        # Кэшируем, так как будем пробегаться по RDD несколько раз
        prediction_and_labels_rdd.cache()
        n_users = prediction_and_labels_rdd.count()
        
        if n_users == 0:
            print("! Нет пользователей для оценки в режиме", mode)
            prediction_and_labels_rdd.unpersist()
            return ZERO_METRICS
        
        # Считаем ranking-метрики через RankingMetrics
        ranking_metrics = RankingMetrics(prediction_and_labels_rdd)
        precision_at_k = ranking_metrics.precisionAt(k)
        recall_at_k = ranking_metrics.recallAt(k)
        map_at_k = ranking_metrics.meanAveragePrecisionAt(k)
        ndcg_at_k = ranking_metrics.ndcgAt(k)
        
        elapsed = time.time() - start_time
        
        print(f"Пользователей оценено: {n_users}")
        print(f"  Precision@{k}: {precision_at_k:.8f}")
        print(f"  Recall@{k}:    {recall_at_k:.8f}")
        print(f"  MAP@{k}:       {map_at_k:.8f}")
        print(f"  NDCG@{k}:      {ndcg_at_k:.8f}")
        print(f"Время: {elapsed:.2f} сек")
        
        prediction_and_labels_rdd.unpersist()
        
        return {
            "k": k,
            "n_users": n_users,
            "precision_at_k": precision_at_k,
            "recall_at_k": recall_at_k,
            "map_at_k": map_at_k,
            "ndcg_at_k": ndcg_at_k
        }
    
    # Считаем оба варианта и возвращаем в одном словаре
    metrics_all  = _compute_single_mode("all")
    metrics_warm = _compute_single_mode("warm")
    
    return {
        "all": metrics_all,
        "warm": metrics_warm
    }

als_ranking_small = compute_ranking_metrics_als_both(
    best_als_small, train_small, test_small, k=10
)

RANKING-МЕТРИКИ ALS @K=10 | mode = 'all'


/usr/local/lib/python3.11/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Пользователей оценено: 598
  Precision@10: 0.00083612
  Recall@10:    0.00022951
  MAP@10:       0.00027240
  NDCG@10:      0.00083373
Время: 3.01 сек
RANKING-МЕТРИКИ ALS @K=10 | mode = 'warm'
Пользователей оценено: 598
  Precision@10: 0.00083612
  Recall@10:    0.00023536
  MAP@10:       0.00027240
  NDCG@10:      0.00083373
Время: 2.43 сек


### Интерпретация ranking-метрик ALS

Для оценки качества рекомендаций ALS были рассчитаны **ranking-метрики в двух режимах**:
- `"all"` — учитываются все релевантные фильмы в тестовой выборке, включая фильмы, которые отсутствовали в обучающей выборке (cold items).
- `"warm"` — учитываются только релевантные фильмы, присутствующие в обучающей выборке, то есть те фильмы, которые модель ALS способна рекомендовать в принципе.

В обоих режимах значения `Precision@10`, `Recall@10`, `MAP@10` и `NDCG@10` оказались **крайне низкими** (менее 0.001), а различия между режимами оказались незначительными.

Это объясняется следующими **факторами**:
1. Датасет MovieLens является сильно разреженным: у большинства пользователей в тестовой выборке имеется лишь 1–2 релевантных фильма, что делает задачу попадания в них в топ-10 довольно сложной.
2. Модель ALS оптимизируется по метрике RMSE, то есть по точности предсказания рейтингов, а не по качеству ранжирования рекомендаций.
3. Даже для “тёплых” фильмов, присутствующих в обучающей выборке, модель часто присваивает более высокие predicted-оценки другим фильмам, которые пользователь в тесте не оценивал.

### Примеры рекомендаций ALS

In [9]:
def show_sample_recommendations(
    model,
    movies_df,
    n_users: int = 3,
    n_recommendations: int = 5
):
    """
    Выводит примеры рекомендаций.
    Args:
        model: Обученная ALS-модель
        movies_df: DataFrame со справочником фильмов
        n_users: Количество пользователей, для которых выводим примеры
        n_recommendations: Количество рекомендаций на пользователя
    Returns:
        List[int]: список userId, для которых были показаны примеры рекомендаций
    """
    print_header("ПРИМЕРЫ РЕКОМЕНДАЦИЙ ALS")

    # Получаем рекомендации для всех пользователей
    all_recs = model.recommendForAllUsers(n_recommendations)
    # Берём первых n_users
    sample_recs = all_recs.limit(n_users).collect()
    # Список userId, для которых выводим примеры
    sample_user_ids = [row["userId"] for row in sample_recs]
    # Локальный справочник movieId -> title
    movies_local = {
        row["movieId"]: row["title"]
        for row in movies_df.select("movieId", "title").collect()
    }
    
    # Выводим рекомендации
    for user_row in sample_recs:
        user_id = user_row["userId"]
        recommendations = user_row["recommendations"]
        print(f"\nПользователь {user_id}:")
        print("-" * 100)
        for i, rec in enumerate(recommendations, 1):
            movie_id = rec["movieId"]
            score = rec["rating"]  # это предсказанный "рейтинг" ALS
            title = movies_local.get(movie_id, "Unknown")
            print(f"  {i}. [{movie_id:<6}] {title[:70]:<70} (score: {score:.3f})")
        
    return sample_user_ids

# Выводим рекомендации и сохраняем список пользователей, для которых выводили рекомендации
sample_user_ids = show_sample_recommendations(
    best_als_small,
    movies_small,
    n_users=3,
    n_recommendations=5
)

ПРИМЕРЫ РЕКОМЕНДАЦИЙ ALS

Пользователь 1:
----------------------------------------------------------------------------------------------------
  1. [7842  ] Dune (2000)                                                            (score: 6.349)
  2. [3379  ] On the Beach (1959)                                                    (score: 6.189)
  3. [6086  ] I, the Jury (1982)                                                     (score: 5.737)
  4. [5915  ] Victory (a.k.a. Escape to Victory) (1981)                              (score: 5.710)
  5. [7025  ] Midnight Clear, A (1992)                                               (score: 5.696)

Пользователь 2:
----------------------------------------------------------------------------------------------------
  1. [5075  ] Waydowntown (2000)                                                     (score: 5.715)
  2. [32892 ] Ivan's Childhood (a.k.a. My Name is Ivan) (Ivanovo detstvo) (1962)     (score: 5.082)
  3. [7842  ] Dune (2000)              

### Объяснение полученных рекомендаций
В модели ALS предсказанный рейтинг вычисляется как скалярное произведение векторов пользователя и фильма:

$$
\hat{r}_{u,i} = \mathbf{p}_u^\top \mathbf{q}_i
$$

При обучении модель **не ограничивается диапазоном** исходной шкалы рейтингов $[0.5; 5]$ и просто минимизирует ошибку (например, RMSE). Поэтому для некоторых пар \((u, i)\) значение скалярного произведения может оказаться больше 5 (или меньше 0.5). Это не ошибка, а следствие того, что модель оптимизирует точность предсказаний, а не строгое соблюдение границ шкалы: значения вроде $\hat{r}_{u,i} > 5$ можно интерпретировать как “очень высокая ожидаемая симпатия пользователя к фильму”.

## Шаг 7. Item-based Collaborative Filtering
---

In [10]:
def build_item_based_cf(
    ratings_df,
    min_cooc_count: int = MIN_COOC_COUNT,
    top_n_neighbors: int = TOP_N_NEIGHBORS
):
    """
    Построение Item-based CF модели.
    Args:
        ratings_df: DataFrame с рейтингами
        min_cooc_count: Минимум общих пользователей для сходства
        top_n_neighbors: Сколько соседей хранить на фильм
    Returns:
        DataFrame с парами (movieId1, movieId2, similarity, cooc_count)
    """
    print_header("ПОСТРОЕНИЕ ITEM-BASED CF")
    print(f"Параметры:")
    print(f"  min_cooc_count:  {min_cooc_count}")
    print(f"  top_n_neighbors: {top_n_neighbors}")

    start_time = time.time()
    
    # Нормализация рейтингов (центрирование по пользователю)
    # Вычисляем средний рейтинг каждого пользователя через оконную функцию
    user_window = Window.partitionBy("userId")
    ratings_centered = (
        ratings_df
        .withColumn(
            "avg_user_rating",
            F.avg("rating").over(user_window)  # Средний рейтинг пользователя
        )
        .withColumn(
            "rating_centered",
            F.col("rating") - F.col("avg_user_rating")  # Центрированный рейтинг
        )
        .select("userId", "movieId", "rating_centered")
    )
    # Кэшируем, т.к. будем использовать дважды в self-join
    ratings_centered = ratings_centered.cache()

    # Для каждого пользователя соединяем все пары его оценённых фильмов    
    # Переименовываем столбцы для join
    ratings_left = ratings_centered.select(
        F.col("userId"),
        F.col("movieId").alias("movie1"),
        F.col("rating_centered").alias("rating1")
    )
    ratings_right = ratings_centered.select(
        F.col("userId"),
        F.col("movieId").alias("movie2"),
        F.col("rating_centered").alias("rating2")
    )
    # Self-join: находим все пары фильмов, оценённых одним пользователем
    pairs = (
        ratings_left
            .join(ratings_right, on="userId", how="inner")
            .filter(F.col("movie1") < F.col("movie2")) # Исключает дубликаты и пары (A, A)
    )

    # Агрегация для косинусного сходства
    similarity_df = (
        pairs
            .groupBy("movie1", "movie2")
            .agg(
                F.sum(F.col("rating1") * F.col("rating2")).alias("dot_product"),  # Сумма произведений
                F.sum(F.col("rating1") ** 2).alias("norm1_sq"),                   # Сумма квадратов первого рейтинга
                F.sum(F.col("rating2") ** 2).alias("norm2_sq"),                   # Сумма квадратов второго рейтинга
                F.count("*").alias("cooc_count")                                  # Число общих пользователей
            )
            # Фильтруем по минимальному числу общих пользователей
            .filter(F.col("cooc_count") >= min_cooc_count)
            # Вычисляем косинусное сходство
            .withColumn(
                "similarity",
                F.col("dot_product") / (
                    F.sqrt(F.col("norm1_sq")) * F.sqrt(F.col("norm2_sq"))
                )
            )
            # Обрабатываем деление на ноль (если все рейтинги одинаковые)
            .withColumn(
                "similarity",
                F.when(F.col("similarity").isNull(), 0.0)
                 .otherwise(F.col("similarity"))
            )
            .select("movie1", "movie2", "similarity", "cooc_count")
    )

    # Отбор top-N соседей для каждого фильма
    # Для movie1: ранжируем movie2 по similarity
    window_movie1 = Window.partitionBy("movie1").orderBy(F.desc("similarity"))    
    neighbors_for_movie1 = (
        similarity_df
            .withColumn("rank", F.row_number().over(window_movie1))
            .filter(F.col("rank") <= top_n_neighbors)
            .select(
                F.col("movie1").alias("movieId"),
                F.col("movie2").alias("neighborId"),
                "similarity",
                "cooc_count"
            )
    )
    # Для movie2: ранжируем movie1 по similarity
    window_movie2 = Window.partitionBy("movie2").orderBy(F.desc("similarity"))
    neighbors_for_movie2 = (
        similarity_df
            .withColumn("rank", F.row_number().over(window_movie2))
            .filter(F.col("rank") <= top_n_neighbors)
            .select(
                F.col("movie2").alias("movieId"),
                F.col("movie1").alias("neighborId"),
                "similarity",
                "cooc_count"
            )
    )
    
    # Объединяем и убираем дубликаты
    item_similarity = (
        neighbors_for_movie1
            .union(neighbors_for_movie2)
            .dropDuplicates(["movieId", "neighborId"])
    )
    # Кэшируем результат
    item_similarity = item_similarity.cache()
    
    elapsed = time.time() - start_time
    n_similarities = item_similarity.count()
    n_movies_with_neighbors = item_similarity.select("movieId").distinct().count()
    print(f"\n  Пар сходства:           {n_similarities:,}")
    print(f"  Фильмов с соседями:     {n_movies_with_neighbors:,}")
    print(f"  Время построения:       {elapsed:.1f} сек")
    
    # Освобождаем промежуточный кэш
    ratings_centered.unpersist()
    
    return item_similarity


# Строим Item-based CF на обучающих данных
item_similarity_small = build_item_based_cf(train_small)

ПОСТРОЕНИЕ ITEM-BASED CF
Параметры:
  min_cooc_count:  5
  top_n_neighbors: 50


25/12/10 20:08:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/12/10 20:08:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/12/10 20:08:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
                                                                                


  Пар сходства:           227,428
  Фильмов с соседями:     3,160
  Время построения:       0.2 сек


### Генерация рекомендаций Item-based CF

In [11]:
def generate_itembased_recommendations(
    item_similarity,
    user_ratings,
    test_users,
    k: int = TOP_K_RECOMMENDATIONS
):
    """
    Генерирует топ-K рекомендаций для пользователей.
    Args:
        item_similarity: DataFrame с парами (movieId, neighborId, similarity)
        user_ratings: DataFrame с центрированными рейтингами пользователей
        test_users: DataFrame с userId для которых нужны рекомендации
        k: Количество рекомендаций
    Returns:
        DataFrame с (userId, movieId, score)
    """
    print_header(f"ГЕНЕРАЦИЯ РЕКОМЕНДАЦИЙ ITEM-BASED CF (топ-{k})")
    start_time = time.time()
    
    # Центрируем рейтинги пользователей
    user_window = Window.partitionBy("userId")
    user_ratings_centered = (
        user_ratings
            .withColumn("avg_user_rating", F.avg("rating").over(user_window))
            .withColumn("rating_centered", F.col("rating") - F.col("avg_user_rating"))
            .select("userId", "movieId", "rating_centered", "avg_user_rating")
    )
    
    # Фильтруем только пользователей из test_users
    user_ratings_filtered = user_ratings_centered.join(test_users, on="userId", how="inner")
    
    # Соединяем рейтинги пользователя с матрицей сходства
    # Для каждого оценённого фильма находим его соседей
    joined = (
        user_ratings_filtered
            .join(
                item_similarity,
                user_ratings_filtered["movieId"] == item_similarity["neighborId"],
                how="inner"
            )
            .select(
                user_ratings_filtered["userId"],
                item_similarity["movieId"].alias("candidate_movieId"),
                user_ratings_filtered["rating_centered"],
                user_ratings_filtered["avg_user_rating"],
                item_similarity["similarity"]
            )
    )
    
    # Исключаем фильмы, которые пользователь уже смотрел
    already_rated = user_ratings_filtered.select("userId", "movieId")
    candidates = (
        joined
            .join(
                already_rated,
                (joined["userId"] == already_rated["userId"]) & 
                (joined["candidate_movieId"] == already_rated["movieId"]),
                how="left_anti"  # Исключаем совпадения
            )
    )
    
    # Агрегируем: вычисляем взвешенный score
    scores = (
        candidates
            .groupBy("userId", "candidate_movieId")
            .agg(
                F.sum(F.col("similarity") * F.col("rating_centered")).alias("weighted_sum"),
                F.sum(F.abs(F.col("similarity"))).alias("sim_sum"),
                F.first("avg_user_rating").alias("avg_user_rating")
            )
            .withColumn(
                "predicted_rating",
                F.col("avg_user_rating") + F.col("weighted_sum") / F.col("sim_sum")
            )
            .filter(F.col("sim_sum") > 0)  # Избегаем деления на ноль
    )
    
    # Отбираем топ-K для каждого пользователя
    window_user = Window.partitionBy("userId").orderBy(F.desc("predicted_rating"))
    recommendations = (
        scores
            .withColumn("rank", F.row_number().over(window_user))
            .filter(F.col("rank") <= k)
            .select(
                "userId",
                F.col("candidate_movieId").alias("movieId"),
                "predicted_rating",
                "rank"
            )
    )
    
    recommendations = recommendations.cache()
    n_recs = recommendations.count()
    n_users_with_recs = recommendations.select("userId").distinct().count()
    
    elapsed = time.time() - start_time
    
    print(f"Сгенерировано рекомендаций: {n_recs:,}")
    print(f"Пользователей с рекомендациями: {n_users_with_recs:,}")
    print(f"Время: {elapsed:.1f} сек")
    
    return recommendations


# Генерируем рекомендации Item-based CF для тестовых пользователей
test_users_small = test_small.select("userId").distinct()
itembased_recs_small = generate_itembased_recommendations(
    item_similarity_small,
    train_small,
    test_users_small
)

ГЕНЕРАЦИЯ РЕКОМЕНДАЦИЙ ITEM-BASED CF (топ-10)


Сгенерировано рекомендаций: 6,100
Пользователей с рекомендациями: 610
Время: 3.8 сек


### RMSE для Item-based CF

In [12]:
def evaluate_itembased_rmse(
    item_similarity,
    train_df,
    test_df
) -> float:
    """
    Оценивает RMSE предсказаний Item-based CF на тестовых данных.
    Args:
        item_similarity: DataFrame с матрицей сходства
        train_df: Обучающий DataFrame
        test_df: Тестовый DataFrame
    Returns:
        RMSE
    """
    print_header("ОЦЕНКА RMSE ITEM-BASED CF")

    start_time = time.time()
    
    # Центрируем train рейтинги
    user_window = Window.partitionBy("userId")
    train_centered = (
        train_df
        .withColumn("avg_user_rating", F.avg("rating").over(user_window))
        .withColumn("rating_centered", F.col("rating") - F.col("avg_user_rating"))
    )

    # Для каждой пары (user, movie) из теста предсказываем рейтинг
    # Соединяем тестовые записи с train через матрицу сходства
    predictions = (
        test_df
            .alias("test")
            .join(
                item_similarity.alias("sim"),
                F.col("test.movieId") == F.col("sim.movieId"),
                how="inner"
            )
            .join(
                train_centered.alias("train"),
                (F.col("test.userId") == F.col("train.userId")) & 
                (F.col("sim.neighborId") == F.col("train.movieId")),
                how="inner"
            )
            .groupBy(
                F.col("test.userId").alias("userId"),
                F.col("test.movieId").alias("movieId"),
                F.col("test.rating").alias("actual_rating")
            )
            .agg(
                F.sum(F.col("sim.similarity") * F.col("train.rating_centered")).alias("weighted_sum"),
                F.sum(F.abs(F.col("sim.similarity"))).alias("sim_sum"),
                F.first("train.avg_user_rating").alias("avg_user_rating")
            )
            .filter(F.col("sim_sum") > 0)
            .withColumn(
                "prediction",
                F.col("avg_user_rating") + F.col("weighted_sum") / F.col("sim_sum")
            )
    )
    
    # Вычисляем RMSE
    rmse_df = (
        predictions
            .withColumn(
                "squared_error",
                (F.col("actual_rating") - F.col("prediction")) ** 2
            )
            .agg(F.sqrt(F.avg("squared_error")).alias("rmse"))
    )
    
    rmse = rmse_df.collect()[0]["rmse"]
    n_predictions = predictions.count()
    
    elapsed = time.time() - start_time
    
    print(f"Предсказаний: {n_predictions:,}")
    print(f"RMSE: {rmse:.6f}")
    print(f"Время: {elapsed:.1f} сек")
    
    return rmse


itembased_rmse_small = evaluate_itembased_rmse(
    item_similarity_small,
    train_small,
    test_small
)

ОЦЕНКА RMSE ITEM-BASED CF
Предсказаний: 16,793
RMSE: 0.899836
Время: 1.4 сек


### Ranking-метрики для Item-based CF

In [13]:
def compute_ranking_metrics_itembased(
    recommendations_df,
    test_df,
    k: int = TOP_K_RECOMMENDATIONS,
    relevance_threshold: float = RELEVANCE_THRESHOLD
) -> dict:
    """
    Вычисляет Ranking-метрики для Item-based CF.
    Args:
        recommendations_df: DataFrame с рекомендациями
        test_df: Тестовый DataFrame
        k: Количество рекомендаций для оценки
        relevance_threshold: Порог релевантности
    Returns:
        Словарь с метриками
    """
    print_header(f"RANKING-МЕТРИКИ ITEM-BASED CF @ K={k}")
    start_time = time.time()
    
    # Ground truth: релевантные фильмы для каждого пользователя
    ground_truth = (
        test_df
            .filter(F.col("rating") >= relevance_threshold)
            .groupBy("userId")
            .agg(F.collect_set("movieId").alias("relevant_movies"))
    )
    
    # Рекомендации в формате списка для каждого пользователя
    recommendations_agg = (
        recommendations_df
            .filter(F.col("rank") <= k)
            .groupBy("userId")
            .agg(
                F.collect_list(
                    F.struct(F.col("rank"), F.col("movieId"))
                ).alias("recs_struct")
            )
            # Сортируем по rank и извлекаем movieId
            .withColumn(
                "recommended_movies",
                F.transform(
                    F.array_sort("recs_struct"),
                    lambda x: x.getField("movieId")
                )
            )
            .select("userId", "recommended_movies")
    )
    
    # Соединяем
    joined = recommendations_agg.join(ground_truth, on="userId", how="inner")
    # Конвертируем в RDD для RankingMetrics
    prediction_and_labels_rdd = (
        joined
        .select("recommended_movies", "relevant_movies")
        .rdd
        .map(lambda row: (list(row.recommended_movies), list(row.relevant_movies)))
    )
    
    prediction_and_labels_rdd.cache()
    n_users_evaluated = prediction_and_labels_rdd.count()
    
    if n_users_evaluated == 0:
        print("! Нет пользователей с релевантными фильмами!")
        return {"precision_at_k": 0, "recall_at_k": 0, "map_at_k": 0, "ndcg_at_k": 0}
    
    # RankingMetrics
    ranking_metrics = RankingMetrics(prediction_and_labels_rdd)
    precision_at_k = ranking_metrics.precisionAt(k)
    recall_at_k = ranking_metrics.recallAt(k)
    map_at_k = ranking_metrics.meanAveragePrecisionAt(k)
    ndcg_at_k = ranking_metrics.ndcgAt(k)
    
    elapsed = time.time() - start_time
    
    print(f"Пользователей оценено: {n_users_evaluated:,}")
    print(f"Metrics@{k}:")
    print(f"  Precision@{k}: {precision_at_k:.6f}")
    print(f"  Recall@{k}:    {recall_at_k:.6f}")
    print(f"  MAP@{k}:       {map_at_k:.6f}")
    print(f"  NDCG@{k}:      {ndcg_at_k:.6f}")
    print(f"Время: {elapsed:.2f} сек")
    
    prediction_and_labels_rdd.unpersist()
    
    return {
        "precision_at_k": precision_at_k,
        "recall_at_k": recall_at_k,
        "map_at_k": map_at_k,
        "ndcg_at_k": ndcg_at_k,
        "k": k,
        "n_users": n_users_evaluated
    }


# Вычисляем Ranking-метрики для Item-based CF
itembased_ranking_small = compute_ranking_metrics_itembased(
    itembased_recs_small,
    test_small
)

RANKING-МЕТРИКИ ITEM-BASED CF @ K=10
Пользователей оценено: 598
Metrics@10:
  Precision@10: 0.003679
  Recall@10:    0.002541
  MAP@10:       0.001331
  NDCG@10:      0.004215
Время: 0.87 сек


### Примеры рекомендаций Item-based CF

In [14]:
def show_sample_recommendations_itembased(
    recommendations_df,
    movies_df,
    user_ids,
    n_recommendations: int = 5
):
    """
    Выводит примеры рекомендаций Item-based CF для заданных пользователей.
    Args:
        recommendations_df: DataFrame с рекомендациями Item-based CF
        movies_df: DataFrame со справочником фильмов
        user_ids: Список userId, для которых нужно показать пример.
        n_recommendations: Количество рекомендаций на пользователя
    """
    print_header("ПРИМЕРЫ РЕКОМЕНДАЦИЙ ITEM-BASED CF")

    # Локальный справочник movieId -> title
    movies_local = {
        row["movieId"]: row["title"]
        for row in movies_df.select("movieId", "title").collect()
    }

    # Выводим рекомендации
    for user_id in user_ids:
        print(f"\nПользователь {user_id}:")
        print("-" * 100)
        user_recs = (
            recommendations_df
            .filter(F.col("userId") == user_id)
            .orderBy("rank")
            .limit(n_recommendations)
            .collect()
        )
        if not user_recs:
            print("    (нет сгенерированных рекомендаций для этого пользователя)")
            continue
        for rec in user_recs:
            movie_id = rec["movieId"]
            score = rec["predicted_rating"]
            rank = rec["rank"]
            title = movies_local.get(movie_id, "Unknown")
            print(f"  {rank}. [{movie_id:<6}] {title[:70]:<70} (predicted: {score:.3f})")

show_sample_recommendations_itembased(
    itembased_recs_small,
    movies_small,
    user_ids=sample_user_ids,
    n_recommendations=5
)

ПРИМЕРЫ РЕКОМЕНДАЦИЙ ITEM-BASED CF

Пользователь 1:
----------------------------------------------------------------------------------------------------
  1. [2548  ] Rage: Carrie 2, The (1999)                                             (predicted: 6.462)
  2. [4649  ] Wet Hot American Summer (2001)                                         (predicted: 5.774)
  3. [51939 ] TMNT (Teenage Mutant Ninja Turtles) (2007)                             (predicted: 5.774)
  4. [8966  ] Kinsey (2004)                                                          (predicted: 5.774)
  5. [255   ] Jerky Boys, The (1995)                                                 (predicted: 5.774)

Пользователь 2:
----------------------------------------------------------------------------------------------------
  1. [707   ] Mulholland Falls (1996)                                                (predicted: 5.160)
  2. [1537  ] Shall We Dance? (Shall We Dansu?) (1996)                               (predicted: 5.160)
 

### Объяснение полученных рекомендаций
В Item-based CF предсказание обычно строится как отклонение от среднего рейтинга пользователя с учётом похожих фильмов:

$$
\hat{r}_{u,i} = \bar{r}_u + \frac{ \sum\limits_{j \in N(i)} s_{i,j} \cdot (r_{u,j} - \bar{r}_u) }{ \sum\limits_{j \in N(i)} |s_{i,j}|}
$$

где  
$\bar{r}_u$ — средний рейтинг пользователя $u$,  
$N(i)$ — множество фильмов, похожих на фильм $i$,  
$s_{i,j}$ — мера сходства между фильмами $i$ и $j$,  
$r_{u,j}$ — рейтинг пользователя $u$ для фильма $j$.

Если пользователь ставил многим похожим фильмам оценки заметно выше своего среднего, числитель может получиться достаточно большим, а итоговое $\hat{r}_{u,i}$ — больше 5. Поскольку формула не ограничивается рамками шкалы $[0.5; 5]$, такие значения являются допустимым результатом и отражают то, что модель ожидает от пользователя “сверхвысокую” оценку этого фильма.

## Шаг 9. Сравнение ALS vs Item-based CF
---

In [15]:
def compare_models(
    als_rmse: float,
    als_ranking: dict,
    itembased_rmse: float,
    itembased_ranking: dict,
) -> pd.DataFrame:
    """
    Сравнивает метрики ALS и Item-based CF.
    Args:
        als_rmse: RMSE модели ALS
        als_ranking: Ranking-метрики ALS
        itembased_rmse: RMSE Item-based CF
        itembased_ranking: Ranking-метрики Item-based CF
    Returns:
        DataFrame со сравнительной таблицей
    """
    k = als_ranking["k"]
    comparison = pd.DataFrame({
        "Metrics": [
            "RMSE",
            f"Precision@{k}",
            f"Recall@{k}",
            f"MAP@{k}",
            f"NDCG@{k}"
        ],
        "ALS": [
            als_rmse,
            als_ranking["precision_at_k"],
            als_ranking["recall_at_k"],
            als_ranking["map_at_k"],
            als_ranking["ndcg_at_k"]
        ],
        "Item-based CF": [
            itembased_rmse,
            itembased_ranking["precision_at_k"],
            itembased_ranking["recall_at_k"],
            itembased_ranking["map_at_k"],
            itembased_ranking["ndcg_at_k"]
        ]
    })
    
    # Добавляем столбец "Лучше"
    comparison["Comparison"] = comparison.apply(
        lambda row: (
            "Equal"  # случай равенства
            if row["ALS"] == row["Item-based CF"]
            else (
                "ALS"
                if (
                    (row["Metrics"] == "RMSE" and row["ALS"] < row["Item-based CF"]) or
                    (row["Metrics"] != "RMSE" and row["ALS"] > row["Item-based CF"])
                )
                else "Item-based CF"
            )
        ),
        axis=1
    )

    # Выводим результат сравнения
    print_header("СРАВНЕНИЕ МОДЕЛЕЙ")
    print(comparison.to_string(index=False))
    
    return comparison


# Сравниваем модели на small-датасете
comparison_small = compare_models(
    als_test_metrics_small["rmse"],
    als_ranking_small["all"], # Используем режим "all", "warm" использовался только для анализа
    itembased_rmse_small,
    itembased_ranking_small,
)

СРАВНЕНИЕ МОДЕЛЕЙ
     Metrics      ALS  Item-based CF    Comparison
        RMSE 0.881034       0.899836           ALS
Precision@10 0.000836       0.003679 Item-based CF
   Recall@10 0.000230       0.002541 Item-based CF
      MAP@10 0.000272       0.001331 Item-based CF
     NDCG@10 0.000834       0.004215 Item-based CF


### Интерпретация результатов сравнения
**По RMSE модель ALS показала лучшее качество**, что означает более точное приближение численных рейтингов.

Однако **по ranking-метрикам значительно лучше оказался Item-based CF**. Это связано с тем, что:
- ALS оптимизируется по RMSE и стремится точно предсказывать рейтинги во всём диапазоне, но не фокусируется на качестве топ-N рекомендаций;
- Item-based CF, напротив, строит рекомендации исходя из похожести на уже понравившиеся фильмы и фактически оптимизирован под задачу "найти ещё несколько релевантных фильмов для пользователя", пусть и ценой менее точных численных предсказаний.

Таким образом, **ALS лучше калиброван как регрессионная модель, тогда как Item-based CF даёт более удачные top-K списки рекомендаций**.

### Освобождаем ресурсы для small-датасета

In [16]:
ratings_small.unpersist()
movies_small.unpersist()
train_small.unpersist()
test_small.unpersist()
item_similarity_small.unpersist()

DataFrame[movieId: int, neighborId: int, similarity: double, cooc_count: bigint]

## Шаг 10. Масштабирование на полный датасет
---
*Моё "железо" не тянет выполнение следующего блока кода, поэтому выполнение этого кода пока что выключено...*

In [17]:
def run_on_full_dataset():
    total_start = time.time()
    
    # Загрузка данных
    data = load_movielens_data(DATA_DIR_FULL, spark)
    validate_data(data)
    ratings_df = data["ratings"]
    movies_df = data["movies"]
    print(f"Размер: {ratings_df.count():,} рейтингов")
        
    # Split
    train_df, test_df = prepare_train_test_split(ratings_df)
    test_df = test_df.cache()
    print(f"Train: {train_df.count():,}, Test: {test_df.count():,}")
    
    # ALS CV
    best_als, cv_model, cv_results = train_als_with_cv(train_df)
    # Оценка ALS
    als_metrics = evaluate_als_on_test(best_als, test_df, dataset_name)
    als_ranking = compute_ranking_metrics_als(best_als, test_df)
    
    # Item-based CF
    item_sim = build_item_based_cf(train_df)
    itembased_rmse = evaluate_itembased_rmse(item_sim, train_df, test_df)
    test_users = test_df.select("userId").distinct()
    itembased_recs = generate_itembased_recommendations(item_sim, train_df, test_users)
    itembased_ranking = compute_ranking_metrics_itembased(itembased_recs, test_df)
    
    # Сравнение
    comparison = compare_models(
        als_metrics["rmse"],
        als_ranking,
        itembased_rmse,
        itembased_ranking,
        dataset_name
    )
    
    total_elapsed = time.time() - total_start
    print(f"\nВремя: {total_elapsed:.1f} сек ({total_elapsed/60:.1f} мин)")
    
    # Освобождаем ресурсы
    ratings_df.unpersist()
    train_df.unpersist()
    test_df.unpersist()
    item_sim.unpersist()

# run_on_full_dataset()

## Шаг 11. Остановка SparkSession
---

In [18]:
spark.stop()

## Выводы
---
В ходе работы были рассмотрены два разных подхода к построению рекомендательных систем: факторизация матрицы рейтингов с помощью ALS и Item-based Collaborative Filtering.

Эксперименты показали, что **ALS** лучше справляется с задачей предсказания численных рейтингов. Более низкое значение RMSE означает, что модель точнее аппроксимирует предпочтения пользователей, используя латентные факторы. Такой подход хорошо подходит в ситуациях, где важно понимать, *насколько* пользователю понравится объект.

В то же время **Item-based Collaborative Filtering** показал себя лучше при оценке качества самих рекомендаций. Более высокие значения ranking-метрик говорят о том, что этот метод чаще включает релевантные фильмы в топ-K рекомендаций. Это объясняется тем, что Item-based CF напрямую опирается на сходство между фильмами и эффективно использует популярные и часто совместно оцениваемые объекты.

В итоге результаты подтверждают, что универсального решения не существует. **ALS** и **Item-based CF** решают задачу рекомендаций по-разному и показывают преимущества в разных сценариях. Выбор конкретного подхода должен зависеть от целей системы — требуется ли более точное предсказание рейтингов или качественные списки рекомендаций для пользователя.